In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
#import seaborn as sns
import pymongo
from pprint import pprint

In [2]:
client = pymongo.MongoClient()
db = client.metrics

### What Experiments Do We Currently Have Data For?

#### CB

In [4]:
cb_collections = \
[str(col)
 for col 
 in sorted(db.collection_names())
 if 
     "CB_" in col and 
     "CR_" in col and 
     "VD" in col and
     "TEST" not in col and 
 1==1]
cb_collections

['CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_VD',
 'CR_CB_SHIFT_REDUCE_PARSER_VD',
 'CR_CB_STACKED_VD-Faulty-IncludesTestCRels',
 'CR_CB_TAGGING_VD_MOST_COMMON_TAG_RNN',
 'CR_CB_TAGGING_VD_RNN']

[str(col)
 for col 
 in sorted(db.collection_names())
 if 
     "CB_" in col and 
     "CR_" in col and 
     "VD" in col and
     "TEST" in col and 
 1==1]

#### SC

In [5]:
sc_collections = \
[str(col)
 for col 
 in sorted(db.collection_names())
 if 
     "SC_" in col and 
     "CR_" in col and 
     "VD" in col and
     "TEST" not in col and 
 1==1]
sc_collections

['CR_SC_STACKED_VD-Faulty-IncludesTestCRels',
 'CR_SC_TAGGING_VD_MOST_COMMON_TAG_RNN']

#### - The 'Stacked' and 'Most Common Tag' Are the Initial 2 Model Types (Tagging without Most Common was a Quick Experiment)

In [6]:
def compute_count_codes(row):

    keys = []
    unique = set()
    for k in row.keys():        
        if "->" in k: #k[0].isdigit():
            keys.append(k)
            unique.add(k)
    assert len(keys) == len(unique), "Duplicate keys found"
    return unique

In [7]:
sc_collections

['CR_SC_STACKED_VD-Faulty-IncludesTestCRels',
 'CR_SC_TAGGING_VD_MOST_COMMON_TAG_RNN']

In [10]:
COLLECTION = [c for c in cb_collections] #if  1==1 or "STACKED" not in c]
#COLLECTION = [c for c in sc_collections] #if  1==1 or "STACKED" not in c]

col2codes = {}
for col in COLLECTION:
    cursor = db[col].find({})
    print(col, cursor.count())
    first_row_codes = None
    for row in cursor:
        codes = compute_count_codes(row)
        if not first_row_codes:
            first_row_codes = set(codes)
            col2codes[col] = codes
        else:            
            assert first_row_codes == codes, "Different codes computed across different runs"

print("")
for col,codes in col2codes.items():
    print(col, len(codes))
            
print("")
for col, codes in col2codes.items():
    for col2, codes2 in col2codes.items():
        if col >= col2:
            continue
        
        if codes != codes2:
            diff_left  = codes - codes2
            diff_right = codes2 - codes

            print("Different codes computed for collections: 1: {col1} [{count_1}] and 2: {col2} [{count_2}]"\
                  .format(col1=col, count_1=len(codes), col2=col2, count_2=len(codes2)))
            
            print("Left Count: {lc} Right Count: {rc}".format(lc=len(diff_left), rc=len(diff_right)))
            print("\tLEFT:  {left}\n\tRIGHT: {right}".format(left="\t".join(diff_left), right="\t".join(diff_right)))
            print("")

CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_VD 1
CR_CB_SHIFT_REDUCE_PARSER_VD 6
CR_CB_STACKED_VD-Faulty-IncludesTestCRels 49
CR_CB_TAGGING_VD_MOST_COMMON_TAG_RNN 6
CR_CB_TAGGING_VD_RNN 6

CR_CB_STACKED_VD-Faulty-IncludesTestCRels 88
CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_VD 82
CR_CB_SHIFT_REDUCE_PARSER_VD 82
CR_CB_TAGGING_VD_RNN 82
CR_CB_TAGGING_VD_MOST_COMMON_TAG_RNN 82

Different codes computed for collections: 1: CR_CB_STACKED_VD-Faulty-IncludesTestCRels [88] and 2: CR_CB_TAGGING_VD_RNN [82]
Left Count: 6 Right Count: 0
	LEFT:  Causer:6->Result:5	Causer:5->Result:11	Causer:Change-factor->Result:50	Causer:7->Result:1	Causer:7->Result:13	Causer:4->Result:Change-factor
	RIGHT: 

Different codes computed for collections: 1: CR_CB_STACKED_VD-Faulty-IncludesTestCRels [88] and 2: CR_CB_TAGGING_VD_MOST_COMMON_TAG_RNN [82]
Left Count: 6 Right Count: 0
	LEFT:  Causer:6->Result:5	Causer:5->Result:11	Causer:Change-factor->Result:50	Causer:7->Result:1	Causer:7->Result:13	Causer:4->Result:Change-factor
	RIG